- **pandas**: Data manipulation and analysis using DataFrames.
- **matplotlib.pyplot**: Create 2D plots and visualizations.
- **sklearn.model_selection.train_test_split**: Split data into training and testing sets.
- **sklearn.tree.DecisionTreeClassifier**: Decision tree classifier model.
- **sklearn.tree.plot_tree**: Visualize the decision tree.
- **sklearn.metrics.accuracy_score**: Calculate model accuracy.
- **sklearn.metrics.classification_report**: Generate classification report with metrics.
- **sklearn.metrics.confusion_matrix**: Compute confusion matrix.
- **seaborn**: Create statistical plots with attractive visuals.


### 1. Preparing and Splitting the Dataset

#### 1.1 Data Reading and Preprocessing

The dataset is first read into a DataFrame. Categorical variables are encoded using `LabelEncoder` to convert them into numerical form, which is necessary for training machine learning models.

##### **Encoding Details**

- **Sex**:
  - Original values: `"Female"` and `"Male"`
  - Encoded values:
    - `0`: Female  
    - `1`: Male  
  - **Encoding method**: `LabelEncoder`

- **Island**:
  - Original values: `"Biscoe"`, `"Dream"`, `"Torgersen"`
  - Encoded values: `0`, `1`, `2` — based on **alphabetical order**
  - **Encoding method**: `LabelEncoder`

All other numerical features (`bill_length_mm`, `bill_depth_mm`, `flipper_length_mm`, `body_mass_g`) are used directly without normalization, as the decision tree algorithm does not require feature scaling.


In [ ]:
!python -m pip install -U scikit-learn
!python -m pip show scikit-learn
!python -c "import sklearn; sklearn.show_versions()"
!python -m pip install matplotlib
!python -m pip install graphviz
!python -m pip install seaborn
!python -m pip install pandas



import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns


data = pd.read_csv('penguins.csv')
data = data.dropna()

feature = data[['island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']]
label = data['species']

feature_encoded = feature.copy()

feature_encoded['island'] = feature_encoded['island'].map({'Biscoe': 0, 'Dream': 1, 'Torgersen': 2})
feature_encoded['sex'] = feature_encoded['sex'].map({'Male': 0, 'Female': 1})




#### 1.2 Visualizing the Original Dataset

The Palmer Penguins dataset contains **344 samples** of three species: **Adelie**, **Chinstrap**, and **Gentoo**, along with features like bill size, flipper length, body mass, and sex.

Some rows have missing values (e.g., in `sex`, `flipper_length_mm`, `bill_length_mm`), so using `dropna()` reduces the dataset to **333 complete samples**.


In [ ]:

plt.figure(figsize=(6,4))
ax = label.value_counts().plot(kind='bar')

label.value_counts().plot(kind='bar')
plt.title("Original Dataset Class Distribution")
plt.xlabel("Species")
plt.ylabel("Count")
plt.tick_params(axis = 'x', rotation = 45)

for p in ax.patches:  
    ax.annotate(f'{p.get_height()}',
                (p.get_x() + p.get_width() / 2., p.get_height()),  
                ha='center', va='center',  
                xytext=(0, 9),  
                textcoords='offset points')
plt.show()


## 1.3 Split dataset
We will split the data according to the ratios 60/40, 40/60, 20/80, 10/90, with a fixed random_state of 0.

In [ ]:
split_ratios = [0.4, 0.6, 0.8, 0.9]
random_state = [0]

datasets = []


for ratio in split_ratios:
    for seed in random_state:
        feature_train, feature_test, label_train, label_test = train_test_split(
            feature_encoded, label,
            train_size = ratio,
            random_state = seed,
            shuffle = True,
            stratify = label,
        )
        
        datasets.append({
            'ratio': ratio,
            'seed': seed,
            'feature_train': feature_train,
            'feature_test': feature_test,
            'label_train': label_train,
            'label_test': label_test,
        })


#### 1.4 Visualizing the Train Dataset and Test Dataset
In the chart, the left bars (in blue) represent the training set distribution, while the right bars (in red) show the test set distribution.

In [ ]:
fig, axes = plt.subplots(len(split_ratios), len(random_state)*2, figsize=(25, 15))

for idx, dataset in enumerate(datasets):
    row = idx // len(random_state)
    col = (idx % len(random_state)) * 2

    # Training set plot
    axes[row, col].bar(dataset['label_train'].value_counts().index, dataset['label_train'].value_counts().values, color='skyblue')
    axes[row, col].set_title(f"Train {int(dataset['ratio']*100)}/{100-int(dataset['ratio']*100)} Seed {dataset['seed']}")
    axes[row, col].tick_params(axis='x', rotation=45)
    axes[row, col].set_ylim(0, 140)

    # Testing set plot
    axes[row, col + 1].bar(dataset['label_test'].value_counts().index, dataset['label_test'].value_counts().values, color='lightcoral')
    axes[row, col + 1].set_title(f"Test {int(dataset['ratio']*100)}/{100-int(dataset['ratio']*100)} Seed {dataset['seed']}")
    axes[row, col + 1].tick_params(axis='x', rotation=45)
    axes[row, col + 1].set_ylim(0, 140)

plt.tight_layout()
plt.show()


### 2. Building the Decision Tree Classifiers
#### 2.1 Training the Decision Tree
We train a **DecisionTreeClassifier** from `sklearn.tree` on each training dataset using the information gain criterion. After training, we visualize the resulting decision tree with **Graphviz**, which provides a clear representation of the model's decision-making process.

To install Graphviz, use the following command:
```bash
pip install graphviz


In [ ]:
from sklearn.tree import export_graphviz
import graphviz

models = []

for i, example in enumerate(datasets):
    # Train model
    model = DecisionTreeClassifier(random_state=example['seed'], criterion='entropy')
    model.fit(example['feature_train'], example['label_train'])
    models.append(model)

    # Export Graphviz data
    dot_data = export_graphviz(
        model,
        out_file=None,
        feature_names=feature_encoded.columns,
        class_names=[str(c) for c in model.classes_],
        filled=True,
        rounded=True,
        special_characters=True
    )

    # Create a Graphviz source object
    graph = graphviz.Source(dot_data)

    # Optional: add custom title to the graph by inserting directly into dot_data
    title = f"Decision Tree {i+1} - Ratio {example['ratio']} Seed {example['seed']}"
    dot_lines = dot_data.split('\n')
    dot_lines.insert(1, f'label="{title}"; labelloc=top; fontsize=20;')  # Add graph label
    graph = graphviz.Source('\n'.join(dot_lines))

    # Display graph
    display(graph)


#### 2.2 Evaluating the Decision Tree Classifiers
After training the decision tree classifiers, it is crucial to assess their performance. We evaluate the models using a variety of metrics, such as accuracy, precision, recall, and F1-score. Additionally, we generate the **confusion matrix** to visualize the classification results and identify areas where the model may be underperforming. These evaluation steps help ensure that the decision tree models are both accurate and reliable for real-world applications.


In [ ]:
for i, model in enumerate(models):
    example = datasets[i]
    X_test = example['feature_test']
    y_test = example['label_test']
    
    # Predict and compute accuracy
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n--- Evaluation for Model {i+1} — Ratio {example['ratio']} — Seed {example['seed']} ---")
    print(f"Accuracy: {acc:.4f}\n")
    
    # Classification report
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=model.classes_, yticklabels=model.classes_)
    plt.title(f"Confusion Matrix (Model {i+1})")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

#### 2.3 The depth and accuracy of a decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import accuracy_score
import graphviz

example = datasets[2]  # datasets[2] is Dataset 3 (80/20)

depths = [None, 2, 3, 4, 5, 6, 7]
accuracy_results = []

for depth in depths:
    model = DecisionTreeClassifier(max_depth=depth, random_state=example['seed'], criterion='entropy')
    model.fit(example['feature_train'], example['label_train'])
    y_pred = model.predict(example['feature_test'])

    acc = accuracy_score(example['label_test'], y_pred)
    accuracy_results.append((depth, acc))

    # Export to Graphviz
    dot_data = export_graphviz(
        model,
        out_file=None,
        feature_names=feature_encoded.columns,
        class_names=[str(c) for c in model.classes_],
        filled=True,
        rounded=True,
        special_characters=True
    )

    # Add title to the Graphviz tree
    title = f"Decision Tree (max_depth={depth}) - Accuracy: {acc:.2f}"
    dot_lines = dot_data.split('\n')
    dot_lines.insert(1, f'label="{title}"; labelloc=top; fontsize=20;')
    graph = graphviz.Source('\n'.join(dot_lines))
    display(graph)

# Print accuracy report
print("\n=== Accuracy Report ===")
print("Max Depth | Accuracy")
for depth, acc in accuracy_results:
    print(f"{str(depth):9} | {acc:.4f}")


In [ ]:


filtered = [(d, a) for d, a in accuracy_results if a is not None]

depths = [str(d) for d, _ in filtered]
accs = [a for _, a in filtered]

plt.figure(figsize=(10, 6), dpi=150)
plt.plot(depths, accs, marker='o')
plt.title("Accuracy vs Max Depth (Palmer Penguins)")
plt.xlabel("Max Depth")
plt.ylabel("Accuracy")
plt.ylim(top=max(accs) + 0.02)

for x, y in zip(depths, accs):
    plt.text(x, y + 0.005, f"{y:.3f}", ha='center', va='bottom', fontsize=9)

plt.grid()
plt.show()

accuracy_table = pd.DataFrame({'max_depth': depths, 'Accuracy': accs})
